# Bagging and Boosting Lab

In this lab we will practice using Bagging and Boosting models on a dataset of your choice.

---

For this lab you should choose datasets in the past that have given you trouble with other models or that you are otherwise interested in looking at with bagging and boosting. The goal is so you get an idea for how they perform compared to more conventional regressions and classifiers.

## 1. Load and inspect the data

Load a dataset or datasets that you have been difficult to get results on in the past, either from labs or projects.

---

### 1.1 Inspect the data and create and X, y  classification and for regression


In [1]:
import pandas as pd
import numpy as np
import patsy

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
datapath = '/Users/tlee010/desktop/dsi-sf-2-timdavidlee/datasets/data_for_diplomas/grad_train.csv'

In [3]:
hs = pd.read_csv(datapath)
hs.head(5)

leaid11        STNAM  FIPST                                leanm11  \
0  1808340      INDIANA     18            Oak Hill United School Corp   
1  4831040        TEXAS     48                           MISSION CISD   
2  4824150        TEXAS     48                         INDUSTRIAL ISD   
3  3612510     NEW YORK     36  GREAT NECK UNION FREE SCHOOL DISTRICT   
4  2803720  MISSISSIPPI     28       POPLARVILLE SEPARATE SCHOOL DIST   

   ALL_COHORT_1112  ALL_RATE_1112  MAM_COHORT_1112 MAM_RATE_1112  \
0              115           92.0              1.0            PS   
1              840           86.0              NaN           NaN   
2               83           95.0              NaN           NaN   
3              618           95.0              NaN           NaN   
4              160           87.0              NaN           NaN   

   MAS_COHORT_1112 MAS_RATE_1112        ...          \
0              NaN           NaN        ...           
1              NaN           NaN        ...           
2              1.0            PS        ...           
3            147.0          GE95        ...           
4              NaN           NaN        ...           

   pct_TEA_Update_Leave_CEN_2010 pct_Census_Mail_Returns_CEN_2010  \
0                          20.98                            75.83   
1                           0.00                            65.85   
2                          31.20                            57.97   
3                           0.00                            61.89   
4                           0.00                            62.66   

   pct_Vacant_CEN_2010 pct_Deletes_CEN_2010  pct_Census_UAA_CEN_2010  \
0                 4.10                 0.25                     8.08   
1                 5.47                 0.00                     9.81   
2                 9.41                 2.05                    10.97   
3                 2.85                 0.00                     5.77   
4                 6.49                 0.00                    16.61   

  pct_Mailback_Count_CEN_2010  pct_FRST_FRMS_CEN_2010  \
0                       87.56                   75.83   
1                       84.72                   59.46   
2                       77.57                   57.97   
3                       91.38                   61.89   
4                       76.90                   57.20   

  pct_RPLCMNT_FRMS_CEN_2010  pct_BILQ_Mailout_count_CEN_2010  \
0                      0.00                             0.00   
1                      6.40                            99.08   
2                      0.00                             0.00   
3                      0.00                             0.00   
4                      5.46                             0.00   

  grad_pct_from_mean  
0          10.791194  
1           3.565682  
2          14.403951  
3          14.403951  
4           4.769934  

[5 rows x 580 columns]

In [13]:
countz = hs.isnull().sum()
exclude = countz[countz>100].index
exclude = [x.encode('utf-8') for x in exclude]
exclude.extend(['leaid11','FIPST','leanm11','MAM_COHORT_1112','School.District','STNAM','MWH_RATE_1112','State_name','County_name','Not_HS_Grad_ACS_08_12', 'Not_HS_Grad_ACSMOE_08_12', 'pct_Not_HS_Grad_ACS_08_12', 'pct_Not_HS_Grad_ACSMOE_08_12','ALL_RATE_1112'])
print exclude


['MAM_COHORT_1112', 'MAM_RATE_1112', 'MAS_COHORT_1112', 'MAS_RATE_1112', 'MBL_COHORT_1112', 'MBL_RATE_1112', 'MHI_COHORT_1112', 'MHI_RATE_1112', 'MTR_COHORT_1112', 'MTR_RATE_1112', 'CWD_COHORT_1112', 'CWD_RATE_1112', 'ECD_COHORT_1112', 'ECD_RATE_1112', 'LEP_COHORT_1112', 'Flag', 'Med_House_value_ACSMOE_08_12', 'Aggr_House_Value_ACS_08_12', 'Aggr_House_Value_ACSMOE_08_12', 'Mail_Return_Rate_CEN_2010', 'pct_Civ_emp_65p_ACS_08_12', 'pct_Civ_emp_65p_ACSMOE_08_12', 'pct_Civ_unemp_65p_ACS_08_12', 'pct_Civ_unemp_65p_ACSMOE_08_12', 'pct_TEA_MailOutMailBack_CEN_2010', 'pct_TEA_Update_Leave_CEN_2010', 'pct_Census_Mail_Returns_CEN_2010', 'pct_Vacant_CEN_2010', 'pct_Deletes_CEN_2010', 'pct_Census_UAA_CEN_2010', 'pct_Mailback_Count_CEN_2010', 'pct_FRST_FRMS_CEN_2010', 'pct_RPLCMNT_FRMS_CEN_2010', 'pct_BILQ_Mailout_count_CEN_2010', 'leaid11', 'FIPST', 'leanm11', 'MAM_COHORT_1112', 'School.District', 'STNAM', 'MWH_RATE_1112', 'State_name', 'County_name', 'Not_HS_Grad_ACS_08_12', 'Not_HS_Grad_ACSMOE_0

In [14]:
import patsy
hsn = hs.drop(exclude,axis=1).copy()
hsn.dropna(inplace=True)
cols = ['Med_HHD_Inc_ACS_08_12','Med_HHD_Inc_ACSMOE_08_12','Aggregate_HH_INC_ACS_08_12','Aggregate_HH_INC_ACSMOE_08_12','Med_House_value_ACS_08_12']
for i in cols:
    hsn[i] = hsn[i].map(lambda x : float(x.replace('$','').replace(',','')))
hsn.columns = [x.replace('.','_').replace(' ','_').replace('$','') for x in hsn.columns]
#print hsn.shape
#print hsn.isnull().sum()
formula = 'grad_pct_from_mean ~' + '+'.join([x for x in hsn.columns if x!='grad_pct_from_mean'])
#print formula

#print hsn.head()

y = hsn['grad_pct_from_mean']
X = hsn[[x for x in hsn.columns if x!='grad_pct_from_mean']]



In [22]:

from sklearn.cross_validation import train_test_split, cross_val_score
Xtrain, Xtest, ytrain,ytest = train_test_split(X,y,test_size = 0.2)

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(Xtrain,ytrain)
print lm.score(Xtest,ytest)

results = pd.DataFrame({'field':X.columns ,'coef':lm.coef_})
results['abs_coef'] = abs(results['coef'])

#print results.sort_values('abs_coef',ascending=False)

cvscores = cross_val_score(lm,Xtrain,ytrain,cv=5)
print np.mean(cvscores), cvscores


0.173978157127
0.194067133171 [ 0.22814344  0.2319013   0.25767163  0.15681795  0.09580134]


In [7]:

# for i,j in zip(hsn.dtypes.index, hsn.dtypes):
#     if j not in ('int64','float64'):
#         print i
        
# hsn[['Med_HHD_Inc_ACS_08_12','Med_HHD_Inc_ACSMOE_08_12','Aggregate_HH_INC_ACS_08_12','Aggregate_HH_INC_ACSMOE_08_12','Med_House_value_ACS_08_12']].head(5)


---

### 2. Decision Tree Regressor

1. Train a decision tree regressor on the regression problem
- Evaluate the score with a 5-fold cross-validation
- How does this compare to the model you fit on this data previously?


In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

dtr = DecisionTreeRegressor()
score = cross_val_score(dtr,Xtrain,ytrain,cv=5,verbose=1)
print np.mean(score), score


0.223485289533 [ 0.18575788  0.22478995  0.23737015  0.28104039  0.18846808]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   11.5s finished


---

### 3. Random Forest Regressor

1. Train a random forest regressor on the regression problem and predict your dependent.
- Evaluate the score with a 5-fold cross-validation
- How does this compare to the model you fit on this data previously? How does it compare to the single decision tree?

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

rfr = RandomForestRegressor(n_estimators=100,max_depth=5)
score = cross_val_score(rfr,Xtrain,ytrain,cv=5,verbose=1)
print np.mean(score), score

0.586325170044 [ 0.56304297  0.60124244  0.59115898  0.60078408  0.57539738]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.6min finished


---

### 4. Extra Trees Regressor

1. Train an extra trees regressor on the regression problem and predict your dependent.
- Evaluate the score with a 5-fold cross-validation
- How does this model compare with the others?

In [28]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.cross_validation import cross_val_score

etr = ExtraTreesRegressor(n_estimators=100,max_depth=5)
score = cross_val_score(etr,Xtrain,ytrain,cv=5,verbose=1)
print np.mean(score), score

0.423075533406 [ 0.3952221   0.45923148  0.44142336  0.39233393  0.42716681]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   25.4s finished


---

### 5. AdaBoost Classifier

1. Train a AdaBoost classifier on your chosen classification problem.
- Evaluate the classifier performance with a 5-fold cross-validation.

In [30]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.cross_validation import cross_val_score

abr = AdaBoostRegressor(n_estimators=100)
score = cross_val_score(abr,Xtrain,ytrain,cv=5,verbose=1)
print np.mean(score), score

0.193950460096 [ 0.1391986   0.23947147  0.21241364  0.25907683  0.11959177]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.3min finished


---

### 6. Gradient Boosted Trees Classifier


1. Train a Gradient Boosting Trees classifier on your chosen classification problem.
- Evaluate the score with a 5-fold cross-validation.
- Compare with the AdaBoost score.

In [32]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import cross_val_score

gbr = GradientBoostingRegressor(n_estimators=100)
score = cross_val_score(gbr,Xtrain,ytrain,cv=5,verbose=1)
print np.mean(score), score

0.6424225408 [ 0.60374014  0.65563246  0.65986442  0.64248068  0.650395  ]


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   55.0s finished


In [37]:
params = {
    'max_depth':[2,4,6,8,10]
    , 'n_estimators':[100]
}
gbr = GradientBoostingRegressor()
from sklearn.grid_search import GridSearchCV

estimator = GridSearchCV(gbr,params, cv=5,verbose=2,n_jobs=-1)
score = estimator.fit(Xtrain,ytrain)



Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] n_estimators=100, max_depth=2 ...................................
[CV] n_estimators=100, max_depth=2 ...................................
[CV] n_estimators=100, max_depth=2 ...................................
[CV] n_estimators=100, max_depth=2 ...................................
[CV] n_estimators=100, max_depth=2 ...................................
[CV] n_estimators=100, max_depth=4 ...................................
[CV] n_estimators=100, max_depth=4 ...................................
[CV] n_estimators=100, max_depth=4 ...................................
[CV] .......................... n_estimators=100, max_depth=2 -  10.9s
[CV] n_estimators=100, max_depth=4 ...................................
[CV] .......................... n_estimators=100, max_depth=2 -  11.1s
[CV] n_estimators=100, max_depth=4 ...................................
[CV] .......................... n_estimators=100, max_depth=2 -  11.3s
[CV] ............

[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  4.6min finished


In [39]:
print estimator.best_params_
print estimator.best_score_

{'n_estimators': 100, 'max_depth': 6}
0.650431259577


In [41]:
gbr = GradientBoostingRegressor(max_depth=6,n_estimators=100, verbose=2)
gbr.fit(Xtrain,ytrain)
scores = gbr.score(Xtest,ytest)

In [42]:
print scores

0.673652240231


### 7. [BONUS] Use gridsearch to fine-tune models of your choice.

1. What are the best parameters found with the gridsearch?
2. How does the best score compare to the model(s) without cross-validation?

**Be careful with many parameters! It can go slow.**